# GSwider Week Coursera Capstone Project

## Battle of the Neighborhoods

## Scarborough, Canada

## Location

Scarborough is a popular destination in Canada as it is said to be one of the most diverse and multicultural areas Toronto. This diverse culture is comparable to New York, due to the amounts of immigration. This trend seems to be continuing, especially since immigration in both the United States and Canada have been increasing.

## Where is Data Coming from?

This project will use FourSquare API as its prime data gathering source. FourSquare has a database of millions of places acquired through crowdsourcing. As a free user of FourSquare’s API, certain features are limited and therefore, certain calls to the API are limited throughout the project. 
Mainly, to compare the similarities of two cities, I am going to explore the neighborhoods, segment them using KMeans, and then group them into clusters to find similar neighborhoods in New York and Toronto.					
This will be done in a Jupyter Notebook, using Python, which requires several libraries such as Pandas, Folium, SciKit Learn, Geopy, and Requests. Pandas will be used to create and manipulate dataframes. Folium will be used to create maps to visualize the neighborhoods. Scikit Learn is the machine learning tool for importing k-means clustering. Geopy will be used to retrieve the location data. Finally, Requests will be used to handle http requests.

## Description of Problem

People moving to different locations are constantly searching for information about their surrounding area. When buying a new home, people always look for what is in proximity to their neighborhood, how much homes in that area cost, nearby schools, crime rates, restaurants, and fun things to do in the area. This project is designed to inform people of the ‘important information’ when determining whether they will be happy with a move to a new location. In this project, I will continue to use the Scarborough data from the previous assignments to analyze and explore the data to make a well-rounded opinion on a particular neighborhood in the Scarborough area, based on key location information from FourSquare.
We can consider a ‘good neighborhood’ as one that has a considerable amount of commercial presence, including dining, entertainment, as well as house pricing, schools, and general safety.


## Part 1: Import Data and Organize Dataframe again from Week 3

#### Import Libraries

In [1]:
import requests
import pandas as pd
import bs4
from bs4 import BeautifulSoup

#### Download URL (link from coursera assignment)

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

#### Create the Dataframe and print the Shape

In [3]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print(df.shape)
df.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Append Latitude and Longitude coordinates to the dataframe

In [4]:
import numpy as np
import geocoder

In [5]:
def get_latlng(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
get_latlng('M5G')

[43.65609081300005, -79.38492999999994]

In [6]:
postal_codes = df['Postalcode']
coords = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]

#### Add Lat and Long columns to original df

In [7]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [8]:
df[df.Postalcode == 'M5G']

,Postalcode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.656091,-79.38493


#### Check new df

In [9]:
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


## Part 2: Clustering

More importing... Again, conda will make this cell run for a bit

In [10]:
import re
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [11]:
address = 'Toronto'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Toronto coordinates are : {}, {}.".format(latitude, longitude))

Toronto coordinates are : 43.653963, -79.387207.


#### Create Map of Toronto

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
map_toronto

#### Add Markers

In [13]:
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'],df['Neighborhood']):
    label='{}, {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='red',
        parse_html=False).add_to(map_toronto)

In [14]:
map_toronto

In [15]:
scarborough_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
address1 = 'Scarborough,Toronto'

geolocator1 = Nominatim()
location1 = geolocator1.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('Scarborough coordinates are: {}, {}.'.format(latitude1, longitude1))

Scarborough coordinates are: 43.773077, -79.257774.


In [16]:
map_scarborough = folium.Map(location=[latitude1, longitude1], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [17]:
#From FourSquare API Lab
CLIENT_ID='QJLI2JYOZQ3SFG5QRQBJTV3S3LMAIDSQIKQTGN0BOR1JO5TO'
CLIENT_SECRET='P4VNCV54FTVULTBXNMJFDHKXNDTN2RO1QBHUBP0BZ5JGBYDW'
VERSION='20180604'

In [18]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude']
neighborhood_longitude = scarborough_data.loc[0, 'Longitude']

neighborhood_name = scarborough_data.loc[0, 'Neighborhood']

print('{} coordinates are : {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, radius, LIMIT)

Rouge, Malvern coordinates are : 43.811525000000074, -79.19551721399995.


In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [21]:
nearby_venues.head()

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649


In [22]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [25]:
scarborough_venues.head(3)
print(scarborough_venues.groupby('Neighborhood').count()[:4])

                                                    Neighborhood Latitude  \
Neighborhood                                                                
Agincourt                                                              13   
Agincourt North, L'Amoreaux East, Milliken, Ste...                      2   
Birch Cliff, Cliffside West                                             6   
Cedarbrae                                                               2   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Agincourt                                                               13   
Agincourt North, L'Amoreaux East, Milliken, Ste...                       2   
Birch Cliff, Cliffside West                                              6   
Cedarbrae                                                                2   

                                                    Venue  Venue Lat

In [26]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()


### Get Top 10 venues per neighborhood

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Shopping Mall,Supermarket,Park,Pool,Chinese Restaurant,Shanghai Restaurant,Skating Rink,Grocery Store,Sushi Restaurant,Bakery
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Women's Store,Hobby Shop,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
2,"Birch Cliff, Cliffside West",Gym Pool,Gym,General Entertainment,Skating Rink,College Stadium,Park,Electronics Store,Gym / Fitness Center,Grocery Store,Golf Course
3,Cedarbrae,Trail,Playground,Women's Store,Discount Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Intersection,Coffee Shop,Soccer Field,Metro Station,Women's Store,Fast Food Restaurant,Gym,Grocery Store
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Pharmacy,Bus Stop,Fried Chicken Joint,Shopping Mall,Coffee Shop,Hobby Shop,Thai Restaurant,Golf Course,Chinese Restaurant
6,"Cliffcrest, Cliffside, Scarborough Village West",Fast Food Restaurant,Discount Store,Liquor Store,Coffee Shop,Furniture / Home Store,Sandwich Place,Pharmacy,Pizza Place,Electronics Store,Gym
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Bakery,Gift Shop,Wine Shop,Women's Store,Electronics Store,Gym Pool,Gym / Fitness Center,Gym,Grocery Store,Golf Course
8,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Coffee Shop,Department Store,Electronics Store,Gym Pool,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop
9,"Guildwood, Morningside, West Hill",Gym / Fitness Center,Park,Construction & Landscaping,Women's Store,Electronics Store,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment


In [47]:
neighborhoods_venues_sorted.count()

Neighborhood              16
1st Most Common Venue     16
2nd Most Common Venue     16
3rd Most Common Venue     16
4th Most Common Venue     16
5th Most Common Venue     16
6th Most Common Venue     16
7th Most Common Venue     16
8th Most Common Venue     16
9th Most Common Venue     16
10th Most Common Venue    16
dtype: int64

### Run K Means to Cluster into 5 Clusters

In [29]:
scarborough_data = scarborough_data.drop(16)
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 2, 0, 4, 0, 0, 0, 0, 0, 0], dtype=int32)

In [30]:
scarborough_merged = scarborough_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [31]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1]).add_to(map_clusters)
       
map_clusters